In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import random

In [2]:
# Node labels
farms = ["F"+str(i) for i in range(1,11)]
custs = ["C"+str(i) for i in range(1,6)]
prods = ['P1', 'P2', 'P3']
dealer = "Dealer"

In [3]:
G = nx.DiGraph()

In [4]:
# Add nodes to network graph
G.add_nodes_from(farms)
G.add_node(dealer)
G.add_nodes_from(prods)
G.add_nodes_from(custs)

In [5]:
farm_dealer_edge_tuples =[(f,dealer) for f in farms]
dealer_prod_edge_tuples =[(dealer, p) for p in prods]
prod_cust_tuples = [(p,c) for p in prods for c in custs]

In [6]:
edges = farm_dealer_edge_tuples + dealer_prod_edge_tuples + prod_cust_tuples

In [7]:
# Connect nodes with edges
G.add_edges_from(edges)

In [8]:
# Add quantities supplied by farms
G['F1']['Dealer']['quantity']=100
G['F2']['Dealer']['quantity']=100
G['F3']['Dealer']['quantity']=100
G['F4']['Dealer']['quantity']=100
G['F5']['Dealer']['quantity']=100
G['F6']['Dealer']['quantity']=100
G['F7']['Dealer']['quantity']=100
G['F8']['Dealer']['quantity']=100
G['F9']['Dealer']['quantity']=100
G['F10']['Dealer']['quantity']=100

In [9]:
def get_total_supply(graph) -> np.int32:
    ''' Return the total number of eggs supplied by farms to dealer'''
    return np.sum([graph[f]['Dealer']['quantity'] for f in graph.predecessors('Dealer')])

In [10]:
supply_qty = get_total_supply(G)
supply_qty

1000

In [11]:
# Set the type of product
G.nodes['P1']['eggs_per_box'] = 6
G.nodes['P2']['eggs_per_box'] = 10
G.nodes['P3']['eggs_per_box'] = 12

In [12]:
# Set the type of customer
G.nodes['C1']['category'] = 'hypermarket'
G.nodes['C2']['category'] = 'supermarket'
G.nodes['C3']['category'] = 'supermarket'
G.nodes['C4']['category'] = 'grocery_store'
G.nodes['C5']['category'] = 'grocery_store'

In [13]:
hypermarket_tupes = [(p,c) for (p,c) in prod_cust_tuples if G.nodes[c]['category']=='hypermarket']
supermarket_tupes = [(p,c) for (p,c) in prod_cust_tuples if G.nodes[c]['category']=='supermarket']
grocery_store_tupes = [(p,c) for (p,c) in prod_cust_tuples if G.nodes[c]['category']=='grocery_store']

In [14]:
# Demand
# Hypermarkets
G['P1']['C1']['demand'] = 200
G['P2']['C1']['demand'] = 100
G['P3']['C1']['demand'] = 50
# Supermarkets
G['P1']['C2']['demand'] = 50
G['P2']['C2']['demand'] = 25
G['P3']['C2']['demand'] = 15
G['P1']['C3']['demand'] = 50
G['P2']['C3']['demand'] = 25
G['P3']['C3']['demand'] = 15
# Grocery store
G['P1']['C4']['demand'] = 50
G['P2']['C4']['demand'] = 0
G['P3']['C4']['demand'] = 0
G['P1']['C5']['demand'] = 50
G['P2']['C5']['demand'] = 0
G['P3']['C5']['demand'] = 0

In [15]:
def get_demand(graph: nx.DiGraph) -> np.array:
    ''' Gets a demand array from graph'''
    global prods
    return np.array([graph[p][c]['demand'] for p in prods for c in graph.successors(p)]).astype(np.int64)   

In [16]:
demand = get_demand(G)
demand

array([200,  50,  50,  50,  50, 100,  25,  25,   0,   0,  50,  15,  15,
         0,   0], dtype=int64)

In [55]:
def feasible_vec(vec:np.array) -> bool:
    '''Returns true if a vec meets demand & supply constraints'''
    global demand, supply_qty, prods
    prod_cap = np.array([G.nodes[p]['eggs_per_box'] for p in prods])
    mat = vec.reshape(len(prods), len(custs))
    supply_check = np.sum(mat * prod_cap[:, None]) <= supply_qty # Check for eggs
    demand_check = np.all((vec <= demand) & (vec >= 0)) # Check boxes
    return demand_check and supply_check

In [25]:
def poss_val(index, val, vec):
    ''' Returns True if the 'val' being placed in 
        'index' position of 'vec' meets 'demand' and 'supply' 
        constraints '''
    vec_copy = vec.copy()
    vec_copy[index]=val
    return feasible_vec(vec_copy)

In [60]:
def get_supply_boxes(vec: np.array) -> np.array:
    ''' Returns an array with boxes of products '''
    global prods, custs
    return np.sum(vec.reshape(len(prods), len(custs)), axis=1)


In [341]:
def random_val(vec:np.array, index: int, graph: nx.DiGraph) -> int:
    ''' Returns a random value that meets demand and supply constraints 
        vec: a vector in which the random value is to placed
        index: the index position in the vector for which the random value is needed
        graph: the supply chain graph '''
    global demand, supply_qty, prods, custs
    
    if demand[index]==0:
        return 0
    else:
        mat = vec.reshape(len(prods), len(custs))
        prod_cap = np.array([G.nodes[p]['eggs_per_box'] for p in prods])
        
        # In-place of unravel index - gets the row, col index if reshaped to matrix
        mat_index = np.arange(0, vec.size).reshape(len(prods), len(custs))
        row, col = np.where(mat_index == index)
        
        alloc_supply = np.sum(mat * prod_cap[:, None])
        available_supply_eggs = supply_qty - (alloc_supply - (vec[index]* prod_cap[row]))
        available_supply_boxes = int(available_supply_eggs // prod_cap[row])
        if  available_supply_boxes and demand[index] > 0:
            return np.random.randint(min(available_supply_boxes, demand[index]))
        else:
            return 0

In [342]:
def random_initiate_vec() -> np.array:
    ''' Returns a vector in the same size of demand that meets demand 
        & supply constraints '''
    global demand
    zero_vec = np.zeros(demand.size)
    indices = np.arange(0,demand.size)
    random.shuffle(indices)
    for i in indices:
        r = random_val(zero_vec, i, G)
        zero_vec[i]=r
    return zero_vec

In [349]:
rv = random_initiate_vec()
rv

array([11., 26., 36.,  8.,  1.,  1.,  7., 17.,  0.,  0.,  7., 14.,  0.,
        0.,  0.])

In [350]:
sup_bo = get_supply_boxes(rv)
sup_bo

array([82., 25., 21.])

In [351]:
feasible_vec(rv)

True

In [313]:
rs = (np.arange(0,demand.shape[0]))
random.shuffle(rs)
rs

array([13,  5,  2,  7,  0, 10,  6,  1, 11, 14, 12,  8,  9,  3,  4])

In [317]:
demand.size

15